### 3. Classifiers

This notebook to classify the probability of fake news traveling from A to B.

In [1]:
ls data

ClassificationModelInput.csv  pol_agg_new.csv
emergent.csv*                 politifact.csv*
fake_localcentralities.csv    politifact_clean.csv
key_mutuality_roshan.csv      real_localcentralities.csv
keys.csv                      snopes.csv*


In [2]:
%pylab inline
import pandas as pd, pyprind

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score,make_scorer

Populating the interactive namespace from numpy and matplotlib


In [3]:
d=pd.read_csv('data/ClassificationModelInput.csv')
sum(sum(d.isnull())) #No NA
print(d.columns)

Index(['Source', 'Destination', 'Monday', 'Elections', 'Military', 'Thursday',
       'Religion', 'Friday', 'Saturday', 'TRUE', 'History', 'Tuesday',
       'Wednesday', 'Sunday', 'page_url', 'Health Care', 'FALSE',
       'jaccard_coeff_connection', 'Neighbor_connection',
       'jaccard_coeff_common_destination', 'Neighbor_common_destination',
       'jaccard_coeff_common_source', 'Neighbor_common_source',
       'SourceSite_outdeg_real', 'DestSite_outdeg_real',
       'SourceSite_outdeg_fake', 'DestSite_outdeg_fake',
       'SourceSite_indeg_fake', 'DestSite_indeg_fake', 'DestSite_indeg_real',
       'SourceSite_indeg_real', 'mutuality_ind', 'PolarityScore'],
      dtype='object')


In [4]:
d.head()

,Source,Destination,Monday,Elections,Military,Thursday,Religion,Friday,Saturday,TRUE,...,SourceSite_outdeg_real,DestSite_outdeg_real,SourceSite_outdeg_fake,DestSite_outdeg_fake,SourceSite_indeg_fake,DestSite_indeg_fake,DestSite_indeg_real,SourceSite_indeg_real,mutuality_ind,PolarityScore
0,www.documentcloud.org,www.politifact.com,0,0,0,0,0,0,0,0,...,0,0.0,6,0.0,2.0,3,1,0.0,0.0,-0.2960
1,jumpinginpools.blogspot.com,townhall.com,3,0,0,0,0,0,0,0,...,0,0.0,9,0.0,0.0,3,0,0.0,0.0,-0.3612
2,jumpinginpools.blogspot.com,pajamasmedia.com,2,0,0,0,0,0,0,0,...,0,0.0,9,0.0,0.0,2,0,0.0,0.0,-0.3612
3,www.billhislam.com,www.nashvillescene.com,2,0,0,0,0,0,0,0,...,0,0.0,6,0.0,0.0,2,0,0.0,0.0,0.7906
4,static.prisonplanet.com,www.healthcare.gov,0,0,0,0,0,2,0,0,...,0,0.0,9,0.0,0.0,2,0,0.0,0.0,0.3182


In [9]:
features=['Monday', 'Elections', 'Military', 'Thursday',
       'Religion', 'Friday', 'Saturday','History', 'Tuesday',
       'Wednesday', 'Sunday', 'page_url', 'Health Care',
       'jaccard_coeff_connection', 'Neighbor_connection',
       'jaccard_coeff_common_destination', 'Neighbor_common_destination',
       'jaccard_coeff_common_source', 'Neighbor_common_source',
       'SourceSite_outdeg_fake', 'DestSite_outdeg_fake',
       'SourceSite_indeg_fake', 'DestSite_indeg_fake', 'mutuality_ind','PolarityScore']

In [15]:
for col in ['Monday', 'Tuesday', 'Wednesday', 
                            'Thursday','Friday', 'Saturday', 'Sunday']:
    mean = d[col].mean()
    std = d[col].std()
    if std != 0:
        d[col] = d[col].apply(lambda row: (row - mean)/std)
        d[col] = d[col].apply(lambda row: math.log(row+2))
    else:
        d[col] = 0
        
d['Label']=d['FALSE'].apply(lambda x: int(x>0))
#d['Label'] = d['FAKE']/(d['TRUE'] + d['FAKE'])
#Check for class imblanace
#print(float(sum(d['Label']==0))/len(d))

#Ready for model
X=d[features]
Y=d['Label']
d['Monday'].head()

0    0.375683
1    1.343641
2    1.339300
3    1.339300
4    0.375683
Name: Monday, dtype: float64

In [16]:
#Random Forest, GBM

#Make AUC the performance metric
models = {'RF':{'model':RandomForestClassifier(),
                     'Params':{'n_estimators':range(50,70,10),
                              'max_depth':range(2,5),
                              'class_weight': 'balanced'}},
                
         'GBM':{'model':GradientBoostingClassifier(),
                     'Params':{'n_estimators':range(50,100,10),
                               'max_depth':range(2,5),
                               'learning_rate':linspace(0.1,0.5,num=20)}}}

In [17]:
print('Number of features:', len(X.columns))
print('Number of records', len(d))

Number of features: 25
Number of records 43


In [20]:
def modeleva(cvfolds):
    allmodels={}
    for model in models.keys():
        grid = GridSearchCV(estimator=models[model]['model'], 
                            param_grid=models[model]['Params'],
                            cv=cvfolds, n_jobs=-1,scoring='roc_auc')
        modelstats={}
        modelstats['model']=grid.fit(X, Y)
        modelstats['best_estimator']=modelstats['model'].best_estimator_
        modelstats['best_score']=modelstats['model'].best_score_
        allmodels[model]=modelstats
        del modelstats
        print(model)
    results=pd.DataFrame.from_dict(allmodels).T.sort_values(by='best_score')
    results=results.reset_index()
    return(results)

In [21]:
allresults=modeleva(5)
allresults

ValueError: Parameter values for parameter (class_weight) need to be a sequence(but not a string) or np.ndarray.

In [35]:
allresults['best_estimator'][0]

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=60, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [41]:
bestmodel

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.10000000000000001, loss='deviance',
              max_depth=4, max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=2,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=40, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)